## 02 Forecasting

### Overview

In [1]:
import numpy as np
import pandas as pd
import ibis
import matplotlib.pyplot as plt
import methods.prep as prep
import methods.vis as vis
import methods.fc as fc
import importlib

SyntaxError: expected ':' (fc.py, line 34)

In [ ]:
importlib.reload(vis)
importlib.reload(prep)

### Load Data

In [ ]:
con = ibis.connect("duckdb://")

In [ ]:
data = ibis.read_csv('data_forecasting/data.csv')

Dev/Test Split

In [ ]:
# Split the data into model development (training + validation) (2007-01-01 to 2019-12-31) 
dev_data = data.filter(data.DATE.year() >= 2007).filter(data.DATE.year() <= 2019)

# and holdout test set (2020-01-01 to the end of the dataset in late 2024)
test_data = data.filter(data.DATE.year() >= 2020)

### Annual Decomposition

To check for seasonality, a look at an annual decomposition of the price of copper futures.

There might well be some seasonal effects in price level, but if so, they seem subtle.

In [ ]:
importlib.reload(prep)
annual_decomp_features = [
    'COPPER_OPEN_NOMINAL'
]
annual_decomp = prep.annual_decomposition(
    dev_data,
    decomp_features=annual_decomp_features
)
annual_decomp.to_pandas().sort_values('DATE').head()

In [ ]:
importlib.reload(vis)
vis.plot_decomp(
    annual_decomp,
    colormap='viridis')

### Stationarity Analysis

For forecasting, features should be stationary, meaning that no significant trends or seasonal patterns should be present in the data.  The mean and variance should be consistent throughout the time period.

No features are stationary without differencing.  The trends are enormous, and while it's difficult to see, some degree of seasonality is almost certainly present.  Many of these look pretty good at first differencing, but we'll want to do additional tests to be confident.

In [ ]:
column_list = [
    'DATE',
    'COPPER_OPEN_NOMINAL',
    'COPPER_OPEN_REAL'
]
df = dev_data.select(column_list).to_pandas()
df['DATE'] = pd.to_datetime(df['DATE'])
df = df.sort_values('DATE')
fig, axes = vis.plot_time_series_diffs(
    df, 
    num_diffs = 2)

### Common Stationarity Tests

##### ADF Test  
Functions and interpretation from [statsmodels](https://www.statsmodels.org/stable/examples/notebooks/generated/stationarity_detrending_adf_kpss.html)

This statistical test checks for a unit root.  If we fail to reject the null hypothesis, the series may be nonstationary.  In this case, the p-value is about 0.19 without differencing and 0.00000 differenced once, suggesting that differencing once is probably appropriate. 

In [ ]:
from statsmodels.tsa.stattools import adfuller

def adf_test(timeseries):
    print("Results of Dickey-Fuller Test:")
    dftest = adfuller(timeseries, autolag="AIC")
    dfoutput = pd.Series(
        dftest[0:4],
        index=[
            "Test Statistic",
            "p-value",
            "#Lags Used",
            "Number of Observations Used",
        ],
    )
    for key, value in dftest[4].items():
        dfoutput["Critical Value (%s)" % key] = value
    print(dfoutput)


In [ ]:
from statsmodels.tsa.stattools import kpss

df = dev_data.to_pandas().sort_values('DATE')[['DATE','COPPER_OPEN_NOMINAL']]
adf_test(df['COPPER_OPEN_NOMINAL'])

In [ ]:
adf_test(df['COPPER_OPEN_NOMINAL'].diff().dropna())

##### KPSS Test
Functions and interpretation from [statsmodels](https://www.statsmodels.org/stable/examples/notebooks/generated/stationarity_detrending_adf_kpss.html)

In this test, the null and alternate hypothesis are reversed: if we reject the null hypothesis, we have evidence that the series is not stationary.  The p-values here are 'higher than 0.01' without differencing and 'higher than 0.1' differenced once, so we can be reasonably confident in rejecting the null hypothesis differencing once.

In [ ]:
from statsmodels.tsa.stattools import kpss

def kpss_test(timeseries):
    print("Results of KPSS Test:")
    kpsstest = kpss(timeseries, regression="c", nlags="auto")
    kpss_output = pd.Series(
        kpsstest[0:3], index=["Test Statistic", "p-value", "Lags Used"]
    )
    for key, value in kpsstest[3].items():
        kpss_output["Critical Value (%s)" % key] = value

    print(kpss_output)

In [ ]:
df = dev_data.to_pandas().sort_values('DATE')[['DATE','COPPER_OPEN_NOMINAL']]
kpss_test(df['COPPER_OPEN_NOMINAL'])

In [ ]:
kpss_test(df['COPPER_OPEN_NOMINAL'].diff().dropna())

### Attempt at Long-Term Models

Key tools:
- [skforecast](https://skforecast.org/)
- [pmdarima](https://github.com/alkaline-ml/pmdarima)
- [sklearn scaling]()

In [ ]:
from skforecast.sarimax import Sarimax

### Train / Validate Split

In [ ]:
# Split the data into training data (2007-2018) 
train_data = dev_data.filter(dev_data.DATE.year() >= 2007).filter(dev_data.DATE.year() <= 2018)

# and validation set (2019 only)
val_data = dev_data.filter(dev_data.DATE.year() == 2019)

In [ ]:
train_df = train_data.to_pandas()
train_df['DATE'] = pd.to_datetime(train_df['DATE'])
train_df = train_df.sort_values('DATE')

In [ ]:
val_df = val_data.to_pandas()
val_df['DATE'] = pd.to_datetime(val_df['DATE'])
val_df = val_df.sort_values('DATE')

##### Pure ARIMA models

PDQ = 1,1,1

In [ ]:

pdq = (1,1,1) # p autoregression lags, d differences, q moving average
model = Sarimax(order = pdq)
model.fit(
    y = train_df['COPPER_OPEN_NOMINAL'])
model.summary()

In [ ]:
pred = model.predict(steps = len(val_df))
pred.head()

In [ ]:
fig, ax=plt.subplots(figsize=(12, 6))

plt.plot(train_df['DATE'],
    train_df['COPPER_OPEN_NOMINAL'],
    label = 'Training')

plt.plot(val_df['DATE'],
    val_df['COPPER_OPEN_NOMINAL'],
    label = 'Validation')

plt.plot(val_df['DATE'],
         pred,
         label = 'Prediction')

ax.legend()

PDQ = 1,2,1

In [ ]:
pdq = (1,2,1) # p autoregression lags, d differences, q moving average
model = Sarimax(order = pdq)
model.fit(y = train_df['COPPER_OPEN_NOMINAL'])
model.summary()

In [ ]:
pred = model.predict(steps = len(val_df))
fig, ax=plt.subplots(figsize=(12, 6))

plt.plot(train_df['DATE'],
    train_df['COPPER_OPEN_NOMINAL'],
    label = 'Training')

plt.plot(val_df['DATE'],
    val_df['COPPER_OPEN_NOMINAL'],
    label = 'Validation')

plt.plot(val_df['DATE'],
         pred,
         label = 'Prediction')

ax.legend()

Simple ARIMAX model

In [ ]:
df = dev_data.to_pandas()
exog_cols = [col for col in df.columns if '_OPEN' in col]
exog = df[exog_cols]

In [ ]:
pdq = (1,1,1) # p autoregression lags, d differences, q moving average
model = Sarimax(order = pdq)
model.fit(
    y = df['COPPER_PRICE'],
    exog = exog)
model.summary()

In [ ]:
df = dev_data.to_pandas()
(df['COPPER_OPEN'] - df['COPPER_OPEN'].mean())/df['COPPER_OPEN'].std()

In [ ]:
df = dev_data.to_pandas()
exog_cols = [col for col in df.columns if '_OPEN' in col]
exog = df[exog_cols]
exog = exog.drop(['NATGAS_OPEN','GOLD_OPEN','CORN_OPEN'], axis='columns')
exog = (exog - exog.mean())/exog.std()

target = df['COPPER_PRICE']
target = (target - target.mean())/target.std()

pdq = (1,1,1) # p autoregression lags, d differences, q moving average
model = Sarimax(order = pdq)
model.fit(
    y = target,
    exog = exog)
model.summary()

### Daily Sliding Window Predictions

These predictions look just one day ahead, using models trained on a shorter window

##### Initial model

In [ ]:
dev_df = dev_data.to_pandas()
dev_df['DATE'] = pd.to_datetime(dev_df['DATE'])
dev_df = dev_df.sort_values('DATE')

In [ ]:
importlib.reload(fc)
dev_df = fc.sliding_window_arima_predictions(
    data = dev_df,
    target_name= 'COPPER_OPEN_NOMINAL',
    pdq = (1,2,1),
    window_size=12
)

In [ ]:
importlib.reload(fc)
dev_df = fc.add_fc_eval_columns(
    df = dev_df,
    pred_feature = 'COPPER_OPEN_NOMINAL'
)

In [ ]:
columns = [
    'DATE',
    
    'COPPER_OPEN_NOMINAL_DELTA_SIGN',
    'COPPER_OPEN_NOMINAL_DELTA_SIGN_PRED',
    'COPPER_OPEN_NOMINAL_DELTA_SIGN_PRODUCT',

    'COPPER_OPEN_NOMINAL_DELTA_ERRVAL',
    'COPPER_OPEN_NOMINAL_DELTA_ERRABS',
    
    'COPPER_OPEN_NOMINAL_DELTA',
    'COPPER_OPEN_NOMINAL_DELTA_PRED',
    
    'COPPER_OPEN_NOMINAL',
    'COPPER_OPEN_NOMINAL_PRED',
]
dev_df[columns].head(20)

##### Model Evaluation

In [ ]:
print("Mean absolute error:  " + str(dev_df['COPPER_OPEN_NOMINAL_ERRABS'].dropna().sum() / dev_df['COPPER_OPEN_NOMINAL_ERRABS'].dropna().count()))

The model is right on the direction of price changes just slightly more often than it's wrong 

In [ ]:
print("Mean product of signs:  " + str(dev_df['COPPER_OPEN_NOMINAL_DELTA_SIGN_PRODUCT'].dropna().sum() / dev_df['COPPER_OPEN_NOMINAL_DELTA_SIGN_PRODUCT'].dropna().count()))

However, when the model is wrong on direction, the prices changes tend to be slightly larger, bringing the expected value of simple directional trading on this model very close to zero.

In [ ]:
expected_trade_value = np.sum(dev_df['COPPER_OPEN_NOMINAL_DELTA_SIGN_PRODUCT'].dropna() * np.abs(dev_df['COPPER_OPEN_NOMINAL_DELTA'].dropna()))
mean_expected_trade_value = expected_trade_value / dev_df['COPPER_OPEN_NOMINAL_DELTA_SIGN_PRODUCT'].dropna().count()

print("Mean expected daily trade yields for single-contract directional trading:  " + str(mean_expected_trade_value))


##### Export Data for Agent Training

In [ ]:
dev_data.to_pandas().to_csv('data_forecasting/dev_data.csv')
test_data.to_pandas().to_csv('data_forecasting/test_data.csv')